In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-03-01'
final_date = '2022-03-31'
query_error_year = 2022
query_error_month = 3

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\03. MARZO\DATA\ERROR MARZO 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 10)
#                 & (query_error['year'] == 2021)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 10 )
#             & (query_error['year'] == 2021), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15 22:01:46,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15 22:02:08,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01 19:53:09,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12 17:07:39,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12 17:07:25,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(76892, 17)

In [5]:
query_error['mes_cierre'].unique()

array([nan, 'Actual'], dtype=object)

In [6]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [7]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [8]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [9]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_6252\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
65839,65839,a890a2d0-14d5-4e57-a0cd-0a8616c3dd76,NEPTUNO,SPQYRVHQ5,2022-03-01,128.41,ERROR,Bank account information not available,2022-03-01 08:19:35,3,2022,NEW,SIN_CLABE,a890a2d0-14d5-4e57-a0cd-0a8616c3dd76,2022-03-01,1,Actual
65314,65314,9eca2c9e-7586-4416-936f-75a768faff41,Cremeria Lety,SBWXML7F8,2022-02-27,64.20,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-03-01 17:37:30,3,2022,NEW,ERROR_USUAL,9eca2c9e-7586-4416-936f-75a768faff41,2022-03-01,1,Actual
65319,65319,3eb75ab3-5471-42d9-b72b-0829d2073bf5,eltrebol dulcerias,MZ367L2W7,2022-02-27,571.57,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:23,3,2022,NEW,ERROR_USUAL,3eb75ab3-5471-42d9-b72b-0829d2073bf5,2022-03-01,1,Actual
65321,65321,2dbdd79f-d96d-4a1d-bcdd-c4bf947eb483,Barbacoa Cathy,M78YS9K3X,2022-02-27,292.27,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:26,3,2022,NEW,ERROR_USUAL,2dbdd79f-d96d-4a1d-bcdd-c4bf947eb483,2022-03-01,1,Actual
65323,65323,f71d063f-0442-46ce-ac47-76cf1ea70cd4,MENU FELIZ,NT6RSVPSQ,2022-02-27,124.57,ERROR,DEV.SPEIFALTA INFORMAC PARA CO,2022-03-01 17:37:24,3,2022,NEW,ERROR_USUAL,f71d063f-0442-46ce-ac47-76cf1ea70cd4,2022-03-01,1,Actual


In [10]:
query_error_current_month.shape

(11866, 17)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# AGRUPADO POR ESQUEMA

In [14]:
group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only      amount                                 \
esquema_query                      ERROR_USUAL            SIN_CLABE             
status_cubeta                            ERROR        NEW     ERROR       NEW   
0                       2022-03-01   11,288.07 450,947.01      0.00 57,429.88   
1                       2022-03-02   49,856.29  50,128.48      1.80 43,117.46   
2                       2022-03-03    1,041.08  52,538.24      0.00 35,915.36   
3                       2022-03-04        0.00   2,909.09      0.00 32,860.58   
4                       2022-03-05        0.48  59,086.27      0.00 37,370.12   

                          
esquema_query        All  
status_cubeta             
0             519,664.96  
1             143,104.03  
2              89,494.68  
3              35,769.67  
4              96,456.87

In [15]:
group_esquema.shape

(32, 6)

In [16]:
writer = pd.ExcelWriter('FASE VI. DATA POLIZA ERROR-SIN CLABE MARZO 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
query_error_current_month.to_excel(writer, sheet_name='DETALLE',index=False)
writer.close()